# Notebook 3 (Diperbarui): Pipeline RAG dan Generasi Jawaban (dengan Azure Embeddings)

Tujuan notebook ini adalah untuk:
1. Memuat vector store (ChromaDB) yang telah diisi dari Notebook 2 (menggunakan Azure Embeddings).
2. Menginisialisasi LLM (Model Bahasa Besar - contoh menggunakan ChatOpenAI standar).
3. Membuat retriever dari vector store.
4. Membangun RAG chain (pipeline) menggunakan LangChain.
5. Mengajukan pertanyaan, mengambil dokumen relevan (dari semua PDF), dan menghasilkan jawaban.
6. Menampilkan sumber dokumen dengan lebih jelas.

# 1. Import library

In [10]:
import os
from dotenv import load_dotenv

from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import ChatOpenAI

from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 2. Load env

In [12]:
load_dotenv()

True

In [13]:
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key_embedding = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
azure_openai_api_version_embedding = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

if not all([azure_openai_endpoint, azure_openai_api_key_embedding, azure_openai_embedding_deployment]):
    print("ERROR: Satu atau lebih variabel environment Azure OpenAI untuk Embeddings (ENDPOINT, API_KEY, EMBEDDING_DEPLOYMENT_NAME) tidak ditemukan.")
    print("Ini diperlukan untuk memuat vector store yang dibuat dengan Azure Embeddings.")
else:
    print("Variabel environment Azure OpenAI Embeddings berhasil dimuat.")

Variabel environment Azure OpenAI Embeddings berhasil dimuat.


In [14]:
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openrouter_model_name = os.getenv("OPENROUTER_MODEL_NAME")

if not openrouter_api_key:
    print("ERROR: OPENROUTER_API_KEY tidak ditemukan. Pastikan file .env sudah benar.")
else:
    print("Variabel environment OpenRouter berhasil dimuat.")

Variabel environment OpenRouter berhasil dimuat.


# 3. Init embedding model & Load vector store

In [15]:
current_notebook_dir = os.getcwd()
project_root = os.path.dirname(current_notebook_dir)

# Sesuaikan jika Anda mengubah nama direktori/koleksi di Notebook 2
vector_store_dir = os.path.join(project_root, "vector_store", "chroma_db_azure_multi")
collection_name = "rag_azure_multi_pdf_collection"

embeddings_model_azure = None # Inisialisasi untuk Azure Embeddings
vector_store = None

if all([azure_openai_endpoint, azure_openai_api_key_embedding, azure_openai_embedding_deployment]):
    try:
        embeddings_model_azure = AzureOpenAIEmbeddings(
            azure_endpoint=azure_openai_endpoint,
            openai_api_key=azure_openai_api_key_embedding,
            azure_deployment=azure_openai_embedding_deployment,
            openai_api_version=azure_openai_api_version_embedding,
        )
        print(f"Model Azure OpenAI Embeddings ('{azure_openai_embedding_deployment}') berhasil diinisialisasi untuk memuat store.")

        if os.path.exists(vector_store_dir) and os.path.exists(os.path.join(vector_store_dir, "chroma.sqlite3")): # Cek lebih spesifik
            print(f"Mencoba memuat ChromaDB dari: {vector_store_dir} dengan koleksi: {collection_name}")
            vector_store = Chroma(
                collection_name=collection_name,
                embedding_function=embeddings_model_azure, # Menggunakan Azure embeddings
                persist_directory=vector_store_dir
            )
            print(f"ChromaDB vector store (dibuat dengan Azure Embeddings) berhasil dimuat.")
        else:
            print(f"ERROR: Direktori/file ChromaDB tidak ditemukan di {vector_store_dir} untuk koleksi {collection_name}.")
            print("Pastikan Anda sudah menjalankan Notebook 2 (yang sudah diupdate dengan Azure) untuk membuat dan mengisi vector store.")

    except Exception as e:
        print(f"Error saat menginisialisasi Azure Embeddings atau memuat vector store: {e}")
else:
    print("Tidak dapat melanjutkan karena konfigurasi Azure OpenAI Embeddings tidak lengkap.")


Model Azure OpenAI Embeddings ('text-embedding-3-small') berhasil diinisialisasi untuk memuat store.
Mencoba memuat ChromaDB dari: D:\Zulfi\CodeLabs\SeiZen\SeiZen-RAG\vector_store\chroma_db_azure_multi dengan koleksi: rag_azure_multi_pdf_collection
ChromaDB vector store (dibuat dengan Azure Embeddings) berhasil dimuat.


# 4. Init LLM

In [16]:
llm = None
if openrouter_api_key: # Menggunakan kunci API OpenRouter untuk LLM
    try:
        llm = ChatOpenAI(
            openai_api_key=openrouter_api_key,
            base_url="https://openrouter.ai/api/v1",
            model_name=openrouter_model_name,
            temperature=0.3,
        )
        print(f"LLM OpenRouter ({llm.model_name}) berhasil diinisialisasi.")
    except Exception as e:
        print(f"Error saat menginisialisasi LLM OpenRouter: {e}")
else:
    print("Tidak dapat menginisialisasi LLM OpenRouter karena API Key tidak ditemukan.")


LLM OpenRouter (deepseek/deepseek-chat-v3-0324:free) berhasil diinisialisasi.


# 5. Creating retriever

In [17]:
retriever = None
if vector_store:
    try:
        retriever = vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 4} # Ambil 4 dokumen teratas untuk konteks yang lebih banyak
        )
        print("Retriever berhasil dibuat dari vector store (yang menggunakan Azure Embeddings).")
    except Exception as e:
        print(f"Error saat membuat retriever: {e}")
else:
    print("Tidak dapat membuat retriever karena vector store tidak tersedia.")

Retriever berhasil dibuat dari vector store (yang menggunakan Azure Embeddings).


# 6. Creating RAG pipeline

In [18]:
rag_chain_lcel = None
if retriever and llm:
    prompt_template_str = """
    You are a very helpful AI assistant. Use the following context to answer user questions. The context comes from various documents. Answer the questions solely based on the provided context. If the information is not available in the context, say you cannot find the answer in the provided documents. Try to keep your answers concise and to the point. If possible and relevant, mention from which source document the information comes (use metadata 'source').

    Context:{context}

    Question:{question}

    Answer (based on the above context):
    """
    prompt = PromptTemplate.from_template(prompt_template_str)

    def format_docs_for_context(docs):
        formatted_docs = []
        for i, doc in enumerate(docs):
            formatted_docs.append(doc.page_content)
        return "\n\n---\n\n".join(formatted_docs)

    rag_chain_lcel = (
        {"question": RunnablePassthrough()}
        | RunnablePassthrough.assign(
            context_docs=lambda x: retriever.invoke(x["question"])
          )
        | RunnablePassthrough.assign(
            context=lambda x: format_docs_for_context(x["context_docs"])
          )
        | prompt
        | llm
        | StrOutputParser()
    )

    print("RAG Chain (LCEL) dengan OpenRouter LLM berhasil dibuat.")

else:
    print("Tidak dapat membuat RAG chain karena retriever atau LLM tidak tersedia.")

RAG Chain (LCEL) dengan OpenRouter LLM berhasil dibuat.


## Testing question

In [21]:
if rag_chain_lcel:
    questions = [
        "What is the correlation of epilepsy stigma in Indonesia?",
        # "Explain self-supervised learning.",
        # "Are there any mentions of 'transformer architecture' in the documents?"
    ]

    for user_question in questions:
        print(f"\n===========================================")
        print(f"Pertanyaan Pengguna: {user_question}")
        print("===========================================")

        try:
            relevant_docs = retriever.invoke(user_question)
            answer = rag_chain_lcel.invoke(user_question)

            print(f"\nJawaban dari RAG Chain (OpenRouter LLM):")
            print(answer)

            if relevant_docs:
                print("\nDokumen Sumber Relevan yang Ditemukan oleh Retriever (menggunakan Azure Embeddings):")
                for i, doc in enumerate(relevant_docs):
                    source_file = doc.metadata.get('source', 'Tidak diketahui')
                    page_number = doc.metadata.get('page', 'N/A')
                    print(f"  {i+1}. Sumber: {source_file}, Halaman: {page_number}")
                    print(f"     Kutipan: {doc.page_content[:150].strip()}...")
            else:
                print("\nTidak ada dokumen sumber relevan yang ditemukan oleh retriever.")

        except Exception as e:
            print(f"Error saat menjalankan RAG chain untuk pertanyaan '{user_question}': {e}")
            import traceback
            traceback.print_exc()
else:
    print("Tidak dapat mengajukan pertanyaan karena RAG chain tidak terinisialisasi.")


Pertanyaan Pengguna: What is the correlation of epilepsy stigma in Indonesia?

Jawaban dari RAG Chain (OpenRouter LLM):
Based on the provided context, the correlation of epilepsy stigma in Indonesia is influenced by the widespread misconceptions and wrong assumptions about the disease. Many Indonesians believe that epilepsy is a contagious, hereditary disease and a curse, which leads to social exclusion, reproach, and suffering for the affected individuals. This stigma is exacerbated by the lack of understanding of the condition's true medical nature (Indian Journal of Public Health Research & Development, March 2020, Vol. 11, No. 03). 

The stigma results in social consequences such as rejection and cleansing of the affected individuals by the community, adding to their burden beyond the medical symptoms of the disease. The high level of stigma is linked to the lack of awareness and education about epilepsy in Indonesia. 

(Source: Indian Journal of Public Health Research & Developme